# ETL Pipeline for Pterodactyl Minecraft Servers Analysis

## Index

- Install requierements
- Import libraries and setup key variables
- Get the Uptime Kuma information
- Load data into the Postgres database

## Install requierements

In [ ]:
!pip install -r requirements.txt
!pip install uptime-kuma-api

## Import libraries and setup key variables
Remember to add you own credentials in the .env file for them to be loaded here

In [ ]:
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
from uptime_kuma_api import UptimeKumaApi, MonitorType

# Load .env file credentials
load_dotenv()

# Database connection
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
dbname = os.getenv('POSTGRES_DBNAME')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
connection = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Kuma connection
kuma_token = os.getenv('TOKEN')
kuma_url = os.getenv('KUMA_URL')
kuma_user = os.getenv('KUMA_USER')
kuma_pass = os.getenv('KUMA_PASS')

# Connect to Kuma
api = UptimeKumaApi('https://kuma.stiv.tech')
#api.login(kuma_user, kuma_pass)
api.login_by_token(kuma_token)

## Get the Uptime Kuma information
Average ping, uptime from 24 hours and 30 days

In [ ]:
# Get the ping result
ping_result = api.avg_ping()

# Get the uptime result
uptime_result = api.uptime()

# Mapping the server names by extracting the id and server names from monitors in kuma and putting them into a dictionary
# (this is because the avg ping and uptime only give us the id and the info, not the server name)
monitors = api.get_monitors()
monitor_dict = {monitor.get("id"): monitor.get("name") for monitor in monitors}

# Replace server numbers with names in the result
ping_result_with_names = {monitor_dict.get(key, f"Unknown Server {key}"): value for key, value in ping_result.items()}
uptime_result_with_names = {monitor_dict.get(key, f"Unknown Server {key}"): value for key, value in uptime_result.items()}
# Convert ping and uptime to DataFrames
ping_df = pd.DataFrame(list(ping_result_with_names.items()), columns=['server_name', 'avg_ping'])
uptime_df = pd.DataFrame(list(uptime_result_with_names.items()), columns=['server_name', 'Uptime'])

# Transform the NaN ping values to 0
ping_df = ping_df.fillna('0')

# Split the 'Uptime' column into '24 Hours' and '30 Days' columns
uptime_df[['uptime_24hours', 'uptime_30days']] = pd.DataFrame(uptime_df['Uptime'].tolist(), index=uptime_df.index)
# Drop the original 'Uptime' column and the '30 days column'
uptime_df = uptime_df.drop(columns=['Uptime', 'uptime_30days'])

# Multiply the percentage columns by 100
#uptime_df['uptime_24hours'] *= 100
#uptime_df['uptime_30days'] *= 100

# Merge the DataFrames and add the 'creation_time' column
final_df = pd.merge(ping_df, uptime_df, on='server_name', how='inner')
final_df['creation_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Filter out rows with 'Unknown Server'
final_df_filtered = final_df[~final_df['server_name'].str.startswith('Unknown Server')]

# Print the filtered DataFrame
final_df_filtered

In [ ]:
# Assuming you already have the final_df_filtered DataFrame

# Convert 'server_name' to string type (necessary for further comparisons)
final_df_filtered['server_name'] = final_df_filtered['server_name'].astype(str)

# Separate the DataFrame into nodes and servers
nodes_df = final_df_filtered[
    final_df_filtered['server_name'].apply(
        lambda x: str(x).split(' | ')[0].isdigit() and int(str(x).split(' | ')[0]) <= 100
    )
]
nodes_df['server_name'] = nodes_df['server_name'].apply(lambda x: str(x).split(' | ')[0])
nodes_df.rename(columns={'server_name': 'node_id'}, inplace=True)

servers_df = final_df_filtered[
    ~final_df_filtered['server_name'].apply(
        lambda x: str(x).split(' | ')[0].isdigit() and int(str(x).split(' | ')[0]) <= 100
    )
]
servers_df['server_name'] = servers_df['server_name'].apply(lambda x: str(x).split(' | ')[0])
servers_df.rename(columns={'server_name': 'server_identifier'}, inplace=True)

# Filter out the row with 'Nodes' in servers_df
servers_df = servers_df[servers_df['server_identifier'] != 'Nodes']

# Print the DataFrames
print("Nodes:")
print(nodes_df)

print("\nServers:")
print(servers_df)

## Load data into the Postgres database
Into kuma table

In [38]:
# Connect to database and upload all new logs into table
SCHEMA = 'kuma'
engine = create_engine(connection)
with engine.connect() as conn:

# Start a new transaction
    trans = conn.begin()

    try:
        # Load all new activity into postgres
        nodes_df.to_sql(name = 'nodes', schema = SCHEMA, con = conn, if_exists='append', index=False)
        servers_df.to_sql(name = 'servers', schema = SCHEMA, con = conn, if_exists='append', index=False)

        # Commit the transaction
        trans.commit()

    except Exception as e:
        # Rollback the transaction on exception
        print('!!! [ERROR IN DATABASE QUERIES] !!!')
        trans.rollback()
        print('Transaction has been rolled back')
        print(f'Error occurred during transaction:\n{e}')
        raise